Google Colab are hosted Jupyter Notebooks with a lot of Power in terms of CPU and RAM. They offer a lot even in the free version, for more can be payed.
You can use different processors for different tasks.

In [1]:
# Importing just as we know it

import pandas as pd

from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf



Instead of local file strucutre, you can use your google drive.

This does not mean, that using files from you local store is not possible. But it may needs extra steps.

In [ ]:
metadata_fp = "/content/drive/MyDrive/Colab Notebooks/ham10000/HAM10000_metadata.csv"
jpgs_fp = "/content/drive/MyDrive/Colab Notebooks/ham10000/jpgs/"
/Users/stefanberkenhoff/Library/CloudStorage/GoogleDrive-stefan.k2ff@googlemail.com/My Drive/Colab Notebooks/ham10000

Working with data in dataframes workes just as with a local file.

In [2]:
metadata = pd.read_csv(metadata_fp)
metadata['image_id'] = metadata['image_id'].apply(lambda x: f"{x}.jpg")

In [3]:
metadata.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset
0,HAM_0000118,ISIC_0027419.jpg,bkl,histo,80.0,male,scalp,vidir_modern
1,HAM_0000118,ISIC_0025030.jpg,bkl,histo,80.0,male,scalp,vidir_modern
2,HAM_0002730,ISIC_0026769.jpg,bkl,histo,80.0,male,scalp,vidir_modern
3,HAM_0002730,ISIC_0025661.jpg,bkl,histo,80.0,male,scalp,vidir_modern
4,HAM_0001466,ISIC_0031633.jpg,bkl,histo,75.0,male,ear,vidir_modern


Training a simple tensor flow model to see which perfomance can be expected.

In [4]:
datagen = ImageDataGenerator(validation_split=0.2) # if you want to split the dataset into training and validation

train_generator = datagen.flow_from_dataframe(
    dataframe=metadata,
    directory=jpgs_fp,
    x_col="image_id", # column name containing the image_ids
    y_col="dx",   # column name for the labels acc to metadata file
    subset="training",
    class_mode="categorical",
    target_size=(128, 128), # or any other size you want
    batch_size=32
)

validation_generator = datagen.flow_from_dataframe(
    dataframe=metadata,
    directory=jpgs_fp,
    x_col="image_id",
    y_col="dx",
    subset="validation",
    class_mode="categorical",
    target_size=(128, 128), # or any other size you want
    batch_size=32
)


Found 8012 validated image filenames belonging to 7 classes.
Found 2003 validated image filenames belonging to 7 classes.


In [5]:
model = tf.keras.models.Sequential([
    # Input layer
    tf.keras.layers.InputLayer(input_shape=(128, 128, 3)),

    # Convolutional block 1
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    # Convolutional block 2
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    # Convolutional block 3
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    # Flattening the output of the convolutional layers
    tf.keras.layers.Flatten(),

    # Dense (fully connected) layers
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),

    # Output layer
    tf.keras.layers.Dense(7, activation='softmax')
])

# Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Summary of the model architecture
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 63, 63, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 30, 30, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 14, 14, 128)       0

In [6]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_generator, validation_data=validation_generator, epochs=10)


Epoch 1/10
251/251 [==============================] - 4052s 16s/step - loss: 3.9087 - accuracy: 0.8173 - val_loss: 5.1334 - val_accuracy: 9.9850e-04
Epoch 2/10
251/251 [==============================] - 90s 358ms/step - loss: 0.5272 - accuracy: 0.8375 - val_loss: 5.8944 - val_accuracy: 9.9850e-04
Epoch 3/10
251/251 [==============================] - 94s 375ms/step - loss: 0.4928 - accuracy: 0.8435 - val_loss: 5.8356 - val_accuracy: 9.9850e-04
Epoch 4/10
251/251 [==============================] - 82s 328ms/step - loss: 0.5142 - accuracy: 0.8439 - val_loss: 6.2038 - val_accuracy: 9.9850e-04
Epoch 5/10
251/251 [==============================] - 82s 327ms/step - loss: 0.4868 - accuracy: 0.8460 - val_loss: 7.1620 - val_accuracy: 0.0020
Epoch 6/10
251/251 [==============================] - 81s 322ms/step - loss: 0.4843 - accuracy: 0.8490 - val_loss: 7.3423 - val_accuracy: 0.0035
Epoch 7/10
251/251 [==============================] - 83s 329ms/step - loss: 0.4828 - accuracy: 0.8467 - val_loss:

In [8]:
from datetime import datetime
timestamp = datetime.now()
model_path = f"/content/gdrive/My Drive/Colab Notebooks/models/model_{timestamp}.keras"
model.save(model_path)
